# Práctica: Análisis Exploratorio de Datos (EDA)

En esta práctica trabajaremos con un dataset que contiene información sobre el rendimiento académico de estudiantes en diferentes asignaturas.

En este punto ya se debe empezar a dominar Matplotlib o Seaborn, por lo que se pide que se intente hacer sin IA en primer lugar para ejercitar la memoria y la búsqueda en la documentación...

El objetivo es realizar un Análisis Exploratorio de Datos (EDA) aplicando:

- Estadística descriptiva
- Visualización con Matplotlib y Seaborn
- Análisis de distribuciones
- Correlaciones
- Comparación entre grupos
- Detección de valores atípicos

No se trata solo de generar gráficos, sino de **interpretar lo que los datos nos están diciendo**. 

__Se deben responder a las preguntas en las celdas markdown__

Si utilizas funciones y parámetros no vistos en los apuntes haz un listado en una celda al final del notebook documentando para que sirve.

# PARTE 1 - Carga y exploración inicial

En 3 celdas de código:

1. Carga el dataset y muestra las primeras filas.
2. Muestra y Analiza:
   - Número de filas y columnas.
   - Tipos de variables.
   - Valores nulos y duplicados
3. Almacena en 2 listas las variables categóricas y las variables numéricas


### Responde a las preguntas:

- ¿Cuáles son las variables numéricas?
- ¿Cuáles son las categóricas?
- ¿Existen valores nulos?
- ¿Qué variable crees que será la más interesante de analizar?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('exams.csv')

# Quick inspection
print(df.info())
print(df.describe())

In [ ]:
# Información general (filas, columnas y tipos de datos)
print("--- Información General ---")
df.info()

# Comprobar valores nulos
print("\n--- Valores Nulos ---")
print(df.isnull().sum())

# Comprobar duplicados
print("\n--- Valores Duplicados ---")
print(f"Total de duplicados: {df.duplicated().sum()}")

In [ ]:
# Identificar numéricas y categóricas automáticamente
# select_dtypes es muy útil para esto
vars_numericas = df.select_dtypes(include=['number']).columns.tolist()
vars_categoricas = df.select_dtypes(include=['object']).columns.tolist()

print(f"Variables Numéricas: {vars_numericas}")
print(f"Variables Categóricas: {vars_categoricas}")

# PARTE 2 — Análisis Variables numéricas

## 2.1. Análisis de distribuciones

Para cada una de las tres variables numéricas (en un figure de 3x1):

1. Representa un histograma con Seaborn con la línea que suaviza la distribución.
2. Añade líneas verticales indicando:
   - Media
   - Mediana


### Responde a las preguntas:
- ¿Son simétricas? ¿cuál lo es más y cuál menos?
- ¿Hay sesgo a la derecha o izquierda?
- ¿Media y mediana coinciden?
- ¿Cuál presenta mayor dispersión?
- ¿Alguna parece aproximarse a una distribución normal?

In [ ]:
# Configuramos el estilo y el tamaño del lienzo
sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(3, 1, figsize=(10, 15))

# Lista de columnas numéricas
cols = ['math score', 'reading score', 'writing score']
colors = ['skyblue', 'salmon', 'lightgreen']

for i, col in enumerate(cols):
    # 1. Histograma con KDE (línea suavizada)
    sns.histplot(df[col], kde=True, ax=axes[i], color=colors[i])
    
    # Calculamos media y mediana
    mean_val = df[col].mean()
    median_val = df[col].median()
    
    # 2. Añadir líneas verticales
    axes[i].axvline(mean_val, color='red', linestyle='--', label=f'Media: {mean_val:.2f}')
    axes[i].axvline(median_val, color='blue', linestyle='-', label=f'Mediana: {median_val:.2f}')
    
    axes[i].set_title(f'Distribución de {col.capitalize()}')
    axes[i].legend()

plt.tight_layout()
plt.show()

## 2.2 Medidas de dispersión

Calcula para cada asignatura:

- Min
- Max
- Media
- Mediana (Q2)
- Q1
- Q3
- Desviación estándar
- Rango intercuartílico (IQR)

Redondea valores a 2 decimales máximo.
Mételos en un Dataframe y muestra el resultado.


### Responde a las preguntas:
- ¿En qué asignatura hay mayor variabilidad?
- ¿En cuál la media es menos representativa?

In [ ]:
# 1. Obtenemos las estadísticas básicas con describe y transponemos para mejor lectura
stats = df[vars_numericas].describe().T

# 2. Calculamos el IQR (Q3 - Q1)
stats['IQR'] = stats['75%'] - stats['25%']

# 3. Renombramos y seleccionamos las columnas solicitadas
# Nota: '50%' es la Mediana (Q2)
stats = stats.rename(columns={
    'min': 'Min',
    'max': 'Max',
    'mean': 'Media',
    '50%': 'Mediana (Q2)',
    '25%': 'Q1',
    '75%': 'Q3',
    'std': 'Desv. Estándar'
})

# 4. Seleccionamos el orden final y redondeamos
resultado = stats[['Min', 'Max', 'Media', 'Mediana (Q2)', 'Q1', 'Q3', 'Desv. Estándar', 'IQR']].round(2)

# Mostrar el Dataframe resultante
resultado

# PARTE 3 - Variables categóricas

## 3.1. Variables categóricas

Representa en un figure de 3 columnas todas las variables categóricas:

1. Cada gráfica es un gráfico de barras. Cada barra es una categoría dentro de la variable. La altura es el número de individuos dentro de esa categoría. Puedes usar countplot de SEaborn.
2. Calcula y muestra la moda de cada variable.
3. En 2 celdas adicionales de código, muestra en gráficos de tarta la distribución de parental level of education y race/ethinicty


In [ ]:
# Definimos las variables categóricas (usando la lista que creamos en la Parte 1)
vars_cat = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']

# Creamos el figure (ajustamos a 2 filas x 3 columnas para que quepan las 5 variables)
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten() # Aplanamos para iterar fácilmente

for i, col in enumerate(vars_cat):
    sns.countplot(data=df, x=col, ax=axes[i], palette='viridis', hue=col, legend=False)
    axes[i].set_title(f'Distribución de {col}')
    axes[i].tick_params(axis='x', rotation=45) # Rotamos etiquetas para que se lean bien

# Eliminamos el último eje sobrante (el sexto)
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

# 2. Calcular y mostrar la moda
print("--- Moda de las variables categóricas ---")
for col in vars_cat:
    # mode() devuelve una Serie, tomamos el primer elemento [0]
    print(f"{col}: {df[col].mode()[0]}")


In [ ]:
plt.figure(figsize=(8, 8))
df['parental level of education'].value_counts().plot.pie(autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Distribución por Nivel Educativo de los Padres')
plt.ylabel('') # Eliminamos la etiqueta del eje Y
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
df['race/ethnicity'].value_counts().plot.pie(autopct='%1.1f%%', startangle=140, colors=sns.color_palette('Set3'))
plt.title('Distribución por Grupo Étnico')
plt.ylabel('')
plt.show()

### Responde a las preguntas para cada variable:
- ¿Qué categoría es dominante en cada variable?
- ¿La distribución de cada variable es equilibrada o claramente descompensada?

# PARTE 4- Comparación entre grupos

## 4.1 Comparación de notas según el sexo

Analiza si existen diferencias en las notas de las 3 asignaturas según el género.

1. Representa un boxplot para cada asignatura. En cada asignatura debe haber una caja para cada género (male, female).

### Analiza y responde:
   - Diferencias en mediana
   - Diferencias en dispersión
   - Presencia de outliers

In [ ]:
# Configuramos un figure de 1 fila y 3 columnas
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

# Lista de asignaturas
asignaturas = ['math score', 'reading score', 'writing score']
palette = {'male': 'skyblue', 'female': 'lightpink'}

for i, col in enumerate(asignaturas):
    sns.boxplot(data=df, x='gender', y=col, ax=axes[i], palette=palette, hue='gender', legend=False)
    axes[i].set_title(f'Distribución de {col.capitalize()} por Género')
    axes[i].set_xlabel('Género')
    axes[i].set_ylabel('Puntuación' if i == 0 else '') # Solo ponemos etiqueta en el primer eje

plt.tight_layout()
plt.show()

## 4.2. Comparación de notas según tipo de lunch

En este apartado analizaremos si existe alguna relación entre el tipo de comida que recibe el estudiante (`lunch`) y su rendimiento académico.

Para ello:

1. Representa un diagrama de barras (`barplot`) para cada asignatura:
   - Matemáticas
   - Lectura
   - Escritura

2. En cada gráfico:
   - El eje X debe representar el tipo de lunch.
   - El eje Y debe representar la **media de la nota**.

⚠ Importante:
El gráfico de barras en Seaborn no muestra frecuencias, sino que calcula automáticamente la media de la variable numérica para cada categoría y añade un intervalo de confianza.

### Responde a las preguntas:
   - ¿Existen diferencias claras entre los grupos?
   - ¿Las medias son significativamente distintas (según el IC)?
   - ¿Qué hipótesis podrías plantear sobre la relación entre alimentación y rendimiento?




In [ ]:
# Configuramos el figure
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

asignaturas = ['math score', 'reading score', 'writing score']

for i, col in enumerate(asignaturas):
    # sns.barplot calcula la media por defecto y añade la línea de error (IC)
    sns.barplot(data=df, x='lunch', y=col, ax=axes[i], palette='muted', hue='lunch', legend=False)
    
    axes[i].set_title(f'Media de {col.capitalize()} por tipo de Lunch')
    axes[i].set_xlabel('Tipo de Almuerzo')
    axes[i].set_ylabel('Nota Media' if i == 0 else '')

plt.tight_layout()
plt.show()

## 4.3. Comparación de notas según el curso de preparación

En este apartado analizaremos si haber completado el curso de preparación (`test preparation course`) influye en el rendimiento académico.

Para ello:

1. Representa un **boxplot** para cada asignatura:
   - Matemáticas
   - Lectura
   - Escritura

2. En cada gráfico:
   - El eje X debe mostrar las categorías del curso de preparación (completed / none).
   - El eje Y debe representar la puntuación obtenida.
   - Se debe visualizar la mediana, el rango intercuartílico (IQR) y los posibles valores atípicos.

3. Añade además la **media de cada grupo como un punto rojo** sobre el boxplot para comparar visualmente media y mediana.

### Responde a las preguntas:
   - ¿Las medianas son mayores en el grupo que completó el curso?
   - ¿Existen diferencias en la dispersión de las notas?
   - ¿Hay más valores atípicos en algún grupo?
   - ¿La media y la mediana son similares o hay indicios de sesgo?


In [ ]:
# Configuramos el figure
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

asignaturas = ['math score', 'reading score', 'writing score']

for i, col in enumerate(asignaturas):
    sns.boxplot(
        data=df, 
        x='test preparation course', 
        y=col, 
        ax=axes[i], 
        palette='Set2',
        hue='test preparation course',
        legend=False,
        showmeans=True, # Muestra la media
        meanprops={"marker":"o", "markerfacecolor":"red", "markeredgecolor":"red", "markersize":"8"} # Personaliza la media como punto rojo
    )
    
    axes[i].set_title(f'{col.capitalize()} vs Prep Course')
    axes[i].set_xlabel('Curso de Preparación')
    axes[i].set_ylabel('Puntuación' if i == 0 else '')

plt.tight_layout()
plt.show()

# PARTE 5 — Relaciones entre variables numéricas

## 5.1. Relación entre variables numéricas

1. Representa un scatterplot de SEaborn entre:
   - Math y Reading
   - Reading y Writing
   - Math y Writing
2. Añade línea de regresión con Seaborn.
3. Calcula la correlación de Pearson y que se vea en la legenda.

### Responde a las preguntas
- ¿Es fuerte?¿Es lineal? ¿es positiva o negativa?
- ¿Qué par de variables está más correlacionado?
- ¿Tiene sentido pedagógico esta relación?


In [ ]:
import scipy.stats as stats

# Definimos los pares a comparar
pares = [('math score', 'reading score'), 
         ('reading score', 'writing score'), 
         ('math score', 'writing score')]

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for i, (x_col, y_col) in enumerate(pares):
    # Calcular coeficiente de correlación de Pearson
    r = df[x_col].corr(df[y_col])
    
    # Crear scatterplot con línea de regresión (regplot)
    sns.regplot(data=df, x=x_col, y=y_col, ax=axes[i], 
                scatter_kws={'alpha':0.4}, line_kws={'color':'red'})
    
    axes[i].set_title(f'{x_col} vs {y_col}')
    # Añadimos la r de Pearson a la leyenda usando un "label" manual o título
    axes[i].annotate(f'Pearson r = {r:.2f}', xy=(0.05, 0.9), xycoords='axes fraction', 
                     bbox=dict(boxstyle="round", fc="w"), fontsize=12)

plt.tight_layout()
plt.show()

## 5.2. Matriz de correlación

Representa la matriz de correlación mediante un heatmap.

In [ ]:
# 1. Calculamos la matriz de correlación (solo para columnas numéricas)
corr_matrix = df.select_dtypes(include=['number']).corr()

# 2. Configuramos el tamaño del gráfico
plt.figure(figsize=(8, 6))

# 3. Creamos el heatmap
sns.heatmap(
    corr_matrix, 
    annot=True,       # Muestra los números (coeficientes) dentro de los cuadros
    cmap='coolwarm',  # Escala de color de frío (azul) a calor (rojo)
    fmt=".2f",        # Redondeo a 2 decimales
    linewidths=0.5,   # Espacio entre celdas
    vmin=-1, vmax=1   # Asegura que la escala siempre sea de -1 a 1
)

plt.title('Matriz de Correlación entre Asignaturas')
plt.show()


# PARTE 6 — Percentiles y detección de outliers

## 6.1. Percentiles

Calcula el percentil 90 en matemáticas y responde.
- ¿Qué nota corresponde?
- ¿Cuántos alumnos lo superan?


In [ ]:
# 1. Calcular el valor del percentil 90
p90_math = df['math score'].quantile(0.90)

# 2. Contar cuántos alumnos están estrictamente por encima de ese valor
superan_p90 = df[df['math score'] > p90_math].shape[0]

print(f"Nota del Percentil 90 en Matemáticas: {p90_math}")
print(f"Número de alumnos que superan esa nota: {superan_p90}")

## 6.2 Outliers con IQR
Detecta manualmente los valores atípicos en matemáticas utilizando la regla (la misma que usa Boxplot para marcar los valores fuera de los bigotes):

Q1 − 1.5×IQR  
Q3 + 1.5×IQR

+ Haz el calculo para saber que valor es el límite de cada outliers.
+ Muestra cuántos outliers hay por encima y por debajo
+ Filtra el DF para mostrar las filas que son los outliers de esta asignatura. 

In [ ]:

# 1. Calculamos los cuartiles y el IQR
q1 = df['math score'].quantile(0.25)
q3 = df['math score'].quantile(0.75)
iqr = q3 - q1

# 2. Calculamos los límites teóricos
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

# 3. Identificamos los outliers
outliers_debajo = df[df['math score'] < limite_inferior]
outliers_encima = df[df['math score'] > limite_superior]

# Mostramos resultados de los límites y conteos
print(f"--- Análisis de Outliers (Math Score) ---")
print(f"Q1: {q1} | Q3: {q3} | IQR: {iqr}")
print(f"Límite inferior (Bigote inferior): {limite_inferior}")
print(f"Límite superior (Bigote superior): {limite_superior}")
print("-" * 40)
print(f"Cantidad de outliers por debajo: {len(outliers_debajo)}")
print(f"Cantidad de outliers por encima: {len(outliers_encima)}")

# 4. Filtramos el DF original para mostrar todos los outliers detectados
df_outliers_math = df[(df['math score'] < limite_inferior) | (df['math score'] > limite_superior)]

# Mostrar las filas resultantes
df_outliers_math

# PARTE 7 - Conclusiones finales

## Conclusiones

Redacta un pequeño informe respondiendo:

- ¿Qué asignatura presenta mayor dificultad?
- ¿Existe diferencia real entre géneros?
- ¿Qué variable influye más en el rendimiento?
- ¿Los datos siguen una distribución normal?
- ¿Qué conclusiones educativas podrían extraerse?
